# Ag-Analytics® - DEM Service API
The Ag-Analytics® DEM Service API allows for clipping boundaries to the 10 meter USGS DEM map of the United States. The service consists of a POST request where the user can pass a GeoJSON boundary, desired output projection as an EPSG code, and a resolution in degrees lat/lon. The result of the post request is a file name that is the result of the clip, reproject, and resample.

The file name can be used as the parameter in a GET request to the same endpoint. Doing so will download the file to the specified local path.


In [1]:
import requests
import json
import time
import os
from pandas.io.json import json_normalize
from collections import defaultdict
import pandas as pd
import zipfile, io
from IPython.display import Image

%matplotlib inline
%autosave 0

Autosave disabled


#### Request parameters details
Request URL:  https://agflaskdev.eastus2.cloudapp.azure.com/flask_api/api/v1.0/polaris_service/download_polaris

1). __aoi__ (.tif, .shp, GeoJSON String): Area of interest as either TIFF file, shapefile, or Geojson string containing boundary coordinates. The maximum aoi area must be less than 200 square miles (518 sq km).
       
2). __Projection__ (string): Projection of result GeoTiff.
    
    Formatted as EPSG code, WKT, or PROJ.4.

3). __Resolution__ (Float): Output GeoTIFF resolution.

    Specified in the units of the projection. E.g., 8-meters in WGS84 ~0.0001 degrees.

4). __Elevation_Index__ (True/False as string): Flag to return elevation indices along with DEM Raster when set to true.

    Relative Elevation: Relative elevation of each pixel as compared to entire field. 
    
    Slope: Slope at each point. 
    
    Topographical Position Index: Index of how each pixel compares to a 5 pixel window of surrounding points.
    
    Terrain Ruggedness Index: Measure of how hilly the area of interest is.    
    



### Note about returning elevation indices

Computing Slope, TPI, and TRI are relatively computationally expensive, and therefore increase the response time significantly. If the DEM request takes ~2 seconds, including a elevation indices will increase the time to ~60 seconds. Since the time required increases as a square of time, the maximum allowable area is 1.5 square miles instead of the 200 square miles in the DEM request without elevation indices.

---
## POST Request API Fuction

In [46]:
def DEM_Service(values, files, headers):
    try:
#         url = 'https://agflaskdev.eastus2.cloudapp.azure.com/flask_api/api/v1.0/dem_service/download_dem'
        url = 'http://127.0.0.1:5000/agflask/api/v1.0/dem_service/download_dem'
        
        # Geojson request or request with geotiff/shapefile.
        if files == None:
            response = requests.post(url, data=values).json()
        else:
            response = requests.post(url, files = files, data = values).json()
        
        print(response)
        
        return response
    
    except Exception as e:
        print(e)
        raise e

---
## A. Request with GeoJSON AOI
The API service takes the request in Json format. Below are examples formatted in Json.

In [47]:
gjson_values = {
    "aoi": "{\"type\":\"Feature\",\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[[[-72.3496565, 42.918982], [-72.3492998, 42.9177602], [-72.3472184, 42.9178623], [-72.3476315, 42.9192373], [-72.3496565, 42.918982]]]},\"properties\":{\"OBJECTID\":4944402,\"CALCACRES\":46.15999985,\"CALCACRES2\":null},\"id\":4944402}",
    "Elevation_Index": "True",
    "Legend_Ranges": "20"
}

gjson_files = None

# Basic header pattern.
gjson_headers={'Content-Type':'application/json'}

# Header for using a subscription key.
# headers={'Content-Type':'application/x-www-form-urlencoded','Ocp-Apim-Subscription-Key': '••••••••••••••••••••••••••••••••'}

### API Call with GeoJSON Boundary

In [48]:
gjson_dem_response=DEM_Service(gjson_values, gjson_files, gjson_headers)

{'Features': [{'attributes': {'CellSize': [9.259259269220298e-05, -9.259259269220298e-05], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-72.3496565, 42.91766322592423, -72.34724909258999, 42.9192373', 'Legend': [{'Area': '5.0 %', 'Count': 16, 'CountAllPixels': 320, 'Max': 270.88823242187505, 'Mean': 269.3876586914063, 'Min': 267.8870849609375, 'color': '#6bd54d'}, {'Area': '5.0 %', 'Count': 16, 'CountAllPixels': 320, 'Max': 271.98125915527345, 'Mean': 271.4347457885742, 'Min': 270.88823242187505, 'color': '#76d14d'}, {'Area': '5.0 %', 'Count': 16, 'CountAllPixels': 320, 'Max': 272.85789489746094, 'Mean': 272.41957702636716, 'Min': 271.98125915527345, 'color': '#82ce4d'}, {'Area': '5.0 %', 'Count': 16, 'CountAllPixels': 320, 'Max': 273.60247802734375, 'Mean': 273.23018646240234, 'Min': 272.8578

---
## B. Request with GeoTIFF AOI
The AOI should be provided as GeoTIFF file. 

In [6]:
#Path to raster on your local machine
gtiff_filepath = r"C:\Users\Alex\Documents\Blobs\Harvest_Rasters\e036f472-7a0b-4d77-9ed7-5e3e91cc150f\raster_VRYIELDVOL_0-0001.tif"
gtiff_files = [('File',(open(gtiff_filepath, 'rb')))]
print(gtiff_files)

#Parameters to create index
gtiff_values = {"Elevation_Index": "False",
            "Projection": "EPSG:4326",
            "Resolution": "0.0001"}

gtiff_headers={'Content-Type':'application/json'}

# Header for using a subscription key.
# headers={'content-type': "application/json",'Ocp-Apim-Subscription-Key': "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

[('File', <_io.BufferedReader name='C:\\Users\\Alex\\Documents\\Blobs\\Harvest_Rasters\\e036f472-7a0b-4d77-9ed7-5e3e91cc150f\\raster_VRYIELDVOL_0-0001.tif'>)]


### API Call with GeoTIFF Boundary

In [7]:
gtiff_dem_response=DEM_Service(gtiff_values, gtiff_files, gtiff_headers)

{'Features': [{'attributes': {'CellSize': [0.0001, -0.0001], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-89.7589617109772, 40.45670547253678, -89.7512617109772, 40.463805472536784', 'Legend': [{'Area': '20.05 %', 'Count': 244, 'CountAllPixels': 1217, 'Max': 141.15856018066407, 'Mean': 141.08349151611327, 'Min': 141.0084228515625, 'color': '#6bd54d'}, {'Area': '19.97 %', 'Count': 243, 'CountAllPixels': 1217, 'Max': 141.2390625, 'Mean': 141.19881134033204, 'Min': 141.15856018066407, 'color': '#d5b64d'}, {'Area': '19.97 %', 'Count': 243, 'CountAllPixels': 1217, 'Max': 141.33377990722656, 'Mean': 141.28642120361326, 'Min': 141.2390625, 'color': '#dbc781'}, {'Area': '19.97 %', 'Count': 243, 'CountAllPixels': 1217, 'Max': 147.81399230957032, 'Mean': 144.57388610839843, 'Min': 141.33377990722656, '

---
## C. Request with shapefile AOI
The AOI should be provided as a zipped file containing a .shp, .shx, .dbf, and .prj.

In [9]:
#Path to zipped shapefile directory on your local machine
shp_filepath = r"C:\Users\Alex\Documents\sample_shp.zip"
shp_file = [('File',(open(shp_filepath, 'rb')))]
print(shp_file)

#Parameters to create index
shp_values = {"Elevation_Index": "False",
            "Projection": "EPSG:4326",
            "Resolution": "0.0001",
            "Legend_Ranges": "20"}

shp_headers={'Content-Type':'application/json'}

# Header for using a subscription key.
# headers={'content-type': "application/json",'Ocp-Apim-Subscription-Key': "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

[('File', <_io.BufferedReader name='C:\\Users\\Alex\\Documents\\sample_shp.zip'>)]


### API Call with Shapefile Boundary

In [11]:
shp_dem_response=DEM_Service(shp_values, shp_file, shp_headers)

{'Features': [{'attributes': {'CellSize': [0.0001, -0.0001], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-121.2484646, 45.4417127, -121.2115646, 45.4668127', 'Legend': [{'Area': '5.0 %', 'Count': 4498, 'CountAllPixels': 89951, 'Max': 382.63002014160156, 'Mean': 369.81505584716797, 'Min': 357.0000915527344, 'color': '#6bd54d'}, {'Area': '5.0 %', 'Count': 4497, 'CountAllPixels': 89951, 'Max': 398.3754577636719, 'Mean': 390.5027389526367, 'Min': 382.63002014160156, 'color': '#76d14d'}, {'Area': '5.0 %', 'Count': 4498, 'CountAllPixels': 89951, 'Max': 414.1468200683594, 'Mean': 406.2611389160156, 'Min': 398.3754577636719, 'color': '#82ce4d'}, {'Area': '5.0 %', 'Count': 4497, 'CountAllPixels': 89951, 'Max': 430.31072998046875, 'Mean': 422.22877502441406, 'Min': 414.1468200683594, 'color': '#8eca4d'

---
## Displaying Response

### Displaying Output Image

In [36]:
#################################################################
# Input boundary type from ['geojson', 'geotiff', 'shapefile']
boundary_type = 'geojson' 
#################################################################

if boundary_type == 'geojson':
    dem_response = gjson_dem_response
elif boundary_type == 'geotiff':
    dem_response = gtiff_dem_response
elif boundary_type == 'shapefile':
    dem_response = shp_dem_response

In [37]:
df=defaultdict(list)
for value in dem_response['Features']:
    attributes = value['attributes']
    df['Max'].append(float(attributes['Max']))
    df['Mean'].append(float(attributes['Mean']))
    df['Min'].append(float(attributes['Min']))
    df['pngb64'].append(attributes['pngb64'])

indexdf=pd.DataFrame.from_dict(df)
indexdf

,Max,Mean,Min,pngb64
0,254.87587,179.644409,116.246864,"data:image/png;base64, iVBORw0KGgoAAAANSUhEUgA..."


In [38]:
#Image Legend
df=defaultdict(list)
for product in dem_response['Features']:
    Legend = product['attributes']['Legend']
    
for val in Legend:
    df['Max'].append(float(val['Max']))
    df['Mean'].append(float(val['Mean']))
    df['Min'].append(float(val['Min']))
    df['Color'].append(val['color'])
#     df['Range'] = ['Low','Medium','High']
    
legenddf = pd.DataFrame.from_dict(df)    
legenddf.style.applymap(lambda x:"background-color: %s"%x, subset=['Color'])

,Max,Mean,Min,Color
0,119.176,117.711,116.247,#6bd54d
1,120.229,119.702,119.176,#76d14d
2,121.064,120.647,120.229,#82ce4d
3,122.003,121.533,121.064,#8eca4d
4,124.643,123.323,122.003,#9ac74d
5,136.906,130.774,124.643,#a5c34d
6,151.587,144.246,136.906,#b1c04d
7,166.058,158.822,151.587,#bdbc4d
8,175.049,170.554,166.058,#c9b94d
9,184.126,179.588,175.049,#d5b64d


In [39]:
# IndexImage = indexdf.loc[indexdf['Index'] == 'POLARIS_Soils','pngb64'].iloc[0]
DEMImage = dem_response["Features"][0]['attributes']["pngb64"]
Image(url = DEMImage, width = 500, height = 500)

---
# GET Request

In [49]:
# Specify local path where file will be downloaded.
# local_path = r"C:\<Path to download directory>" # E.g., r"C:\Users\John_Doe\Documents\rasters"
local_path = r'C:\Users\Alex\Documents\Blobs'
# Create GET request payload.
if 'Index_Files' in dem_response.keys():
    files = []
    files.append(dem_response['FileName'])
    for file in dem_response['Index_Files']:
        files.append(file)
    values = {'FileNames': files}
else:
    values = {'FileNames': [dem_response['FileName'],]}
print(values)

{'FileNames': ['result_raster_dem_20200227170844408308.tif', 'raster_RelativeElevation_78916.tif', 'raster_Slope_60540.tif', 'raster_TPI_64215.tif', 'raster_TRI_27819.tif']}


In [54]:
def DEM_get(values, local_path):
    try:
#         url = 'https://agflaskdev.eastus2.cloudapp.azure.com/flask_api/api/v1.0/dem_service/download_dem'
        url = 'http://127.0.0.1:5000/agflask/api/v1.0/dem_service/download_dem'
        
        print('values', values)

        for file in values['FileNames']:
            print('file', file)
            download_path = os.path.join(local_path, file)
            
            file_dict = {'FileName': file}

            response = requests.get(url, params=file_dict)

            open(download_path, 'wb').write(response.content)

            print(response.url)
            
            print(response)
    
    except Exception as e:
        print(e)
        raise e

In [ ]:
DEM_get(values, local_path)

values {'FileNames': ['result_raster_dem_20200227170844408308.tif', 'raster_RelativeElevation_78916.tif', 'raster_Slope_60540.tif', 'raster_TPI_64215.tif', 'raster_TRI_27819.tif']}
file result_raster_dem_20200227170844408308.tif
